In [71]:
%pip install tiktoken
%pip install python-dotenv
%pip install openai

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [36]:
import tiktoken
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

We are going to test multiple prompts by estimating the tokens prior to sending them to the LLM. Then, we will get the actual tokens, and thereafter, we can find the percentage difference to see how far off our estimate is."

In [35]:
prompts = {
    1: "The sky turned a deep shade of orange as the sun dipped below the horizon.",
    2: "She couldn't believe how quickly the day had passed, feeling like it was only minutes ago she had woken up.",
    3: "The cat leaped gracefully onto the windowsill, watching the world outside with curious eyes.",
    4: "He carefully arranged the books on the shelf, ensuring they were organized by size and color.",
    5: "The gentle hum of the air conditioner was the only sound in the otherwise quiet room.",
    6: "They planned to meet at the coffee shop at noon, eager to catch up after months apart.",
    7: "The scent of fresh-baked bread filled the kitchen, making everyone's mouth water.",
    8: "A sudden gust of wind scattered the papers on her desk, sending them flying across the room.",
    9: "He struggled to solve the puzzle, but the answer seemed just out of reach.",
    10: "The old clock in the hallway chimed, marking the start of a new hour."
}


In [50]:
def estimating_tokens(prompt):
    enc = tiktoken.encoding_for_model("gpt-4o")
    estimate_prompt_tokens = len(enc.encode(prompt))
    return int(estimate_prompt_tokens)

In [58]:
def actual_tokens_func(prompt):
    client = OpenAI(
        api_key=os.getenv("OPENAI_API_KEY")
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"user",
                "content":prompt
            }
        ]
    )
    # conversion to json
    json_response = json.loads(response.model_dump_json())
    actual_token_info = json_response['usage']['prompt_tokens']
    return int(actual_token_info)


In [72]:
percentage_off_list =[]

In [73]:

for prompt_number,prompt in (prompts.items()):
    print(f"Estimating tokens {estimating_tokens(prompt)} in prompt number {prompt_number}")
    print(f"Actual tokens {actual_tokens_func(prompt)} in prompt number {prompt_number}")

    estimated_tokens = estimating_tokens(prompt)
    actual_tokens = actual_tokens_func(prompt)

    difference = int(actual_tokens) - int(estimated_tokens)
    percentage_off = int((int(difference) / int(estimated_tokens)) * 100)
    print(percentage_off)
    percentage_off_list.append(percentage_off)

print(percentage_off_list)




    

Estimating tokens 16 in prompt number 1
Actual tokens 23 in prompt number 1
43
Estimating tokens 23 in prompt number 2
Actual tokens 30 in prompt number 2
30
Estimating tokens 18 in prompt number 3
Actual tokens 25 in prompt number 3
38
Estimating tokens 18 in prompt number 4
Actual tokens 25 in prompt number 4
38
Estimating tokens 17 in prompt number 5
Actual tokens 24 in prompt number 5
41
Estimating tokens 19 in prompt number 6
Actual tokens 26 in prompt number 6
36
Estimating tokens 16 in prompt number 7
Actual tokens 23 in prompt number 7
43
Estimating tokens 19 in prompt number 8
Actual tokens 26 in prompt number 8
36
Estimating tokens 16 in prompt number 9
Actual tokens 23 in prompt number 9
43
Estimating tokens 17 in prompt number 10
Actual tokens 24 in prompt number 10
41
[43, 30, 38, 38, 41, 36, 43, 36, 43, 41]


In [78]:
def average_percentage():
    added = 0
    for i in percentage_off_list:
        added += i
    return(added/len(percentage_off_list))


After the results, we can see that we have an average of 38.9 for the estimation.

In [80]:
print(f"AVERAGE PERCENTAGE DIFFERENCE: {average_percentage()}%")

AVERAGE PERCENTAGE DIFFERENCE: 38.9%
